In [49]:
import xspec as x
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from astropy.io import fits



In [50]:

#Load relxill and test it

# --- STEP 1: DEFINE PATHS ---
# Path to the folder where you compiled relxill (contains lmodel.dat)
relxill_dir = "/home/kyle/software/relxill"

# Path to the folder containing the heavy FITS tables (often the same dir)
relxill_tables = "/home/kyle/software/relxill"

# --- STEP 2: SET ENVIRONMENT VARIABLE ---
# This tells relxill where to look for the tables so you don't have to copy them
os.environ["RELXILL_TABLE_PATH"] = relxill_tables

# --- STEP 3: LOAD THE MODEL ---
# This mimics the "lmod relxill /path" command
print("Loading Relxill...")
x.AllModels.lmod("relxill", relxill_dir)

# --- STEP 4: USE IT ---
# Now you can use 'relxill', 'relxillCp', 'relxilllp', etc.
# Example: simple relativistic reflection
m = x.Model("tbabs * relxillcp")

# Verify it loaded
print(m.componentNames)

Loading Relxill...
Model package relxill successfully loaded.

Model TBabs<1>*relxillCp<2> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
                           Data group: 1
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   relxillCp  Incl       deg      30.0000      frozen
   3    2   relxillCp  a                   0.998000     +/-  0.0          
   4    2   relxillCp  Rin                 -1.00000     frozen
   5    2   relxillCp  Rout                400.000      frozen
   6    2   relxillCp  Rbr                 15.0000      frozen
   7    2   relxillCp  Index1              3.00000      frozen
   8    2   relxillCp  Index2              3.00000      frozen
   9    2   relxillCp  z                   0.0          frozen
  10    2   relxillCp  gamma               2.00000      +/-  0.0          
  11    2   relxillCp  logxi               3.10000      +/-  0.0          
  12    2   relxillCp  logN       cm^-3  

In [56]:
# --- 1. SETUP & DATA LOADING ---
# 1. Clear any existing data/models
x.AllData.clear()
x.AllModels.clear()
# 2. Define your file paths for clarity
rgs1_spec = "../products/0865600201/rgs/P0865600201R1S004SRSPEC1001.FIT"
rgs1_o2_spec = "../products/0865600201/rgs/P0865600201R1S004SRSPEC2001.FIT"
# rgs1_spec = "../products/0865600201/rgs/time_intervals/Shallow/rgs1_src_o1_Shallow.fits"

rgs2_spec = "../products/0865600201/rgs/P0865600201R2S005SRSPEC1001.FIT"
rgs2_o2_spec = "../products/0865600201/rgs/P0865600201R2S005SRSPEC2001.FIT"
# rgs2_spec = "../products/0865600201/rgs/time_intervals/Shallow/rgs2_src_o1_Shallow.fits"

# for spec_file in [rgs1_spec, rgs2_spec]:
#     with fits.open(spec_file, mode='update') as hdul:
#         hdul[1].header['POISSERR'] = True
#     print(f"Updated {spec_file}")
#
# for spec_file in [rgs1_spec, rgs2_spec]:
#     with fits.open(spec_file) as hdul:
#         poisserr = hdul[1].header.get('POISSERR', 'NOT FOUND')
#         print(f"{spec_file}: POISSERR = {poisserr}")

rgs1_resp= "../products/0865600201/rgs/P0865600201R1S004RSPMAT1001.FIT"
rgs1_o2_resp = "../products/0865600201/rgs/P0865600201R1S004RSPMAT2001.FIT"
# rgs1_resp=   "../products/0865600201/rgs/time_intervals/Shallow/rgs1_o1_Shallow.rmf"


rgs2_resp = "../products/0865600201/rgs/P0865600201R2S005RSPMAT1001.FIT"
rgs2_o2_resp = "../products/0865600201/rgs/P0865600201R2S005RSPMAT2001.FIT"
# rgs2_resp = "../products/0865600201/rgs/time_intervals/Shallow/rgs2_o1_Shallow.rmf"

In [57]:
# 3. Load data into separate Data Groups (1:1 and 2:2)
# Using the f-string here mimics "data 1:1 file1 2:2 file2"
x.AllData(f"1:1 {rgs1_spec} 2:2 {rgs1_o2_spec}")


# 4. Assign Response Matrices manually
s1 = x.AllData(1) # Handle for RGS1
s2 = x.AllData(2) # Handle for RGS2

s1.response = rgs1_resp
s2.response = rgs1_o2_resp

# Set Global XSPEC Settings
x.Plot.device = "/null"
x.Plot.xAxis = "keV" # Or "angstrom" for RGS
x.Xset.xsect = "vern"
x.Xset.abund = "wilm"

# --- 2. MODEL DEFINITION ---
# Model: constant * tbfeo * (nthcomp + diskbb)
m1 = x.Model("constant * tbfeo * (nthcomp + diskbb + bbodyrad)")
m2 = x.AllModels(2)

# --- 3. APPLYING CORRECTED PARAMETERS FROM LATEST PN FIT ---

# 1. Instrument Normalization (constant)
m1(1).values = 1.0
m1(1).frozen = True
m2(1).values = 0.86781 
m2(1).frozen = False

# 2. Absorption with variable O and Fe (tbfeo)
m1(2).values = 0.28802  # Best-fit nH from PN
m1(2).frozen = True     # Keep fixed to see O/Fe line residuals
m1(3).values = 0.1     # Oxygen abundance
m1(3).frozen = False
m1(4).values = 0.1      # Iron abundance
m1(4).frozen = False
m1(5).values = 0.0      # Redshift
m1(5).frozen = True

# 3. nthComp Continuum
m1(6).values = 1.49188  # Best-fit Gamma from PN
m1(6).frozen = True     # Keep shape fixed
m1(7).values = 1.62383  # Best-fit kT_e from PN
m1(7).frozen = True     # Keep shape fixed
m1(8).link   = "p12"    # kT_bb linked to diskbb Tin (p12)
m1(9).values = 1        # inp_type (diskbb)
m1(9).frozen = True
m1(10).values = 0.0     # Redshift
m1(10).frozen = True
m1(11).values = 0.00224 # Best-fit norm from PN
m1(11).frozen = True   # UNFROZEN to allow for flux scaling

# 4. diskbb Continuum
m1(12).values = 0.71659 # Best-fit Tin from PN
m1(12).frozen = True    # Keep shape fixed
m1(13).values = 0.89052 # Best-fit norm from PN
m1(13).frozen = True   #norm

# --- 4. FITTING ---
x.Fit.statMethod = "cstat"
x.Fit.query = "yes"
x.Fit.perform()


2 spectra  in use
 
Spectral Data File: ../products/0865600201/rgs/P0865600201R1S004SRSPEC1001.FIT  Spectrum 1
Net count rate (cts/s) for Spectrum:1  1.422e-01 +/- 1.293e-03
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-2775
  Telescope: XMM Instrument: RGS1  Channel Type: PI
  Exposure Time: 8.507e+04 sec
 Using fit statistic: cstat
 No response loaded.

Spectral Data File: ../products/0865600201/rgs/P0865600201R1S004SRSPEC2001.FIT  Spectrum 2
Net count rate (cts/s) for Spectrum:2  6.300e-02 +/- 8.606e-04
 Assigned to Data Group 2 and Plot Group 2
  Noticed Channels:  1-2775
  Telescope: XMM Instrument: RGS1  Channel Type: PI
  Exposure Time: 8.507e+04 sec
 Using fit statistic: cstat
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Response successfully loaded.
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Response successfully loaded.
 

In [58]:

# --- NEATLY PRINT BEST FIT OUTPUT (Dual Data Group) ---
print("\n" + "="*95)
print(f"{'GROUP':<5} | {'C#':<3} | {'P#':<3} | {'COMPONENT':<12} | {'PARAMETER':<12} | {'VALUE':<12} | {'ERROR'}")
print("-" * 95)

# Track the global parameter index manually
for group_idx in [1, 2]:
    m = x.AllModels(group_idx)
    global_par_idx = 1
    
    for comp_idx, comp_name in enumerate(m.componentNames, start=1):
        comp = getattr(m, comp_name)
        for par_name in comp.parameterNames:
            par = getattr(comp, par_name)
            
            # Formatting Value and Error string
            val = par.values[0]
            if par.link != "":
                err_str = f"LINKED ({par.link})"
            elif par.frozen:
                err_str = "FROZEN"
            else:
                err_str = f"+/- {par.sigma:.5f}" if par.sigma > 0 else "N/A"
            
            # Print row logic: 
            # Show all of Group 1. 
            # For Group 2, only show things that aren't redundant links (like constant norm)
            if group_idx == 1 or par.link == "" or comp_name == "constant":
                print(f"{group_idx:<5} | {comp_idx:<3} | {global_par_idx:<3} | {comp_name:<12} | {par_name:<12} | {val:<12.5f} | {err_str}")
            
            global_par_idx += 1

print("-" * 95)
print(f"Final {x.Fit.statMethod.upper()}: {x.Fit.statistic:.2f}")
print(f"Degrees of Freedom: {x.Fit.dof}")
print(f"Reduced Statistic: {x.Fit.statistic/x.Fit.dof:.3f}")
print("="*75 + "\n")


GROUP | C#  | P#  | COMPONENT    | PARAMETER    | VALUE        | ERROR
-----------------------------------------------------------------------------------------------
1     | 1   | 1   | constant     | factor       | 1.00000      | FROZEN
1     | 2   | 2   | TBfeo        | nH           | 0.28802      | FROZEN
1     | 2   | 3   | TBfeo        | O            | 0.06299      | +/- 0.05058
1     | 2   | 4   | TBfeo        | Fe           | 0.00000      | +/- 0.25623
1     | 2   | 5   | TBfeo        | redshift     | 0.00000      | FROZEN
1     | 3   | 6   | nthComp      | Gamma        | 1.49188      | FROZEN
1     | 3   | 7   | nthComp      | kT_e         | 1.62383      | FROZEN
1     | 3   | 8   | nthComp      | kT_bb        | 0.71659      | LINKED (= p12)
1     | 3   | 9   | nthComp      | inp_type     | 1.00000      | FROZEN
1     | 3   | 10  | nthComp      | Redshift     | 0.00000      | FROZEN
1     | 3   | 11  | nthComp      | norm         | 0.00224      | FROZEN
1     | 4   | 12  | di

In [73]:

x.Plot.setRebin(5, 20)

x.Plot("data")
# Spectrum 1
energy_x   = x.Plot.x(1)      # X-axis values
energy_err = x.Plot.xErr(1)   # X-axis error bars
data_y     = x.Plot.y(1)      # Data counts/rate
data_err   = x.Plot.yErr(1)   # Data errors
model_y    = x.Plot.model(1)  # The folded model values

# Spectrum 2
energy_x_2   = x.Plot.x(2)
energy_err_2 = x.Plot.xErr(2)
data_y_2     = x.Plot.y(2)
data_err_2   = x.Plot.yErr(2)
model_y_2    = x.Plot.model(2)
y_units = x.Plot.labels()[1]
print(y_units)

x.Plot("delchi")
residuals = x.Plot.y(1)
residuals_2 = x.Plot.y(2)



counts s$^{-1}$ keV$^{-1}$


In [76]:
# python
%matplotlib notebook

plt.figure(figsize=(12, 6), dpi=100)
plt.subplot(2, 1, 1)
plt.gca().set_axisbelow(False)  # ensure artists can be above the grid

# Spectrum 1 with larger points
plt.errorbar(
    energy_x, data_y,
    yerr=data_err, xerr=energy_err,
    fmt='o', ms=1,
    elinewidth=0.8, capsize=2,
    ls='', lw=1, label='rgs 1 data'
)


# Spectrum 2 with smaller points
plt.errorbar(
    energy_x_2, data_y_2,
    yerr=data_err_2, xerr=energy_err_2,
    fmt='o', ms=1,
    elinewidth=0.6, capsize=1,
    color='green', alpha=0.6, ls='', label='rgs 2 data'
)

# Draw models last with higher zorder so they sit above the markers
plt.step(energy_x, model_y, where='mid', color='C1', lw=0.5, zorder=20, label='rgs 1 Model ')
plt.step(energy_x_2, model_y_2, where='mid', color='purple', lw=0.5, zorder=20, label='rgs 2 Model')

plt.ylim(1e-4, None)
# plt.xlim(4, 35)
plt.yscale("log")
plt.ylabel(r"Counts s$^{-1}$ Hz $^{-1}$")
plt.title("Spectrum and Model")
plt.legend(loc="best")

plt.subplot(2, 1, 2)
# plt.xlim(4, 35)
plt.step(energy_x, residuals, where='mid', label='rgs 1 order 1', lw=1.0)
plt.step(energy_x_2, residuals_2, where='mid', label='rgs 1 order 2', color='green', lw=1.0)
plt.axhline(0, color='red', linestyle='--')
plt.legend(loc="best")

plt.xlabel("keV")
plt.ylabel(r"Residuals ($\chi$)")
plt.tight_layout()
plt.show()


/tmp/ipykernel_14520/900770027.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 6), dpi=100)


<IPython.core.display.Javascript object>